In [2]:
%load_ext autoreload
%autoreload 2

import os
import time
import numpy as np
import torch
from tqdm import trange

from game.flappy_bird import FlappyBirdEnv
from agent.dqn_agent import DQNAgent, DQNConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


d:\2025\ML\flappybird_ai\venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [ ]:

env = FlappyBirdEnv(seed=42)
state_dim = 5
action_dim = 2

cfg = DQNConfig(
	state_dim=state_dim,
	action_dim=action_dim,
	gamma=0.99,
	lr=1e-3,
	batch_size=64,
	replay_size=50_000,
	start_learning_after=5_000,
	target_update_freq=1_000,
	eps_start=1.0,
	eps_end=0.05,
	eps_decay_steps=50_000,
	gradient_clip_norm=5.0,
)

agent = DQNAgent(cfg)
print("Using device:", agent.cfg.device)

Using device: cpu


In [4]:
num_episodes = 500  
max_steps_per_ep = 10_000

moving_avg_window = 50
ep_returns = []
losses = []

for ep in trange(num_episodes, desc="Training"):
	state = env.reset()
	ep_return = 0.0

	for t in range(max_steps_per_ep):
		action = agent.select_action(state)
		next_state, reward, done, info = env.step(action)

		agent.store(state, action, reward, next_state, done)
		loss = agent.train_step()
		if loss is not None:
			losses.append(loss)

		ep_return += reward
		state = next_state
		if done:
			break

	ep_returns.append(ep_return)

	# Display simple stats every few episodes
	if (ep + 1) % 10 == 0:
		recent = ep_returns[-moving_avg_window:]
		mavg = np.mean(recent) if recent else 0.0
		print(f"Ep {ep+1}/{num_episodes} | Return: {ep_return:.1f} | 50-ep avg: {mavg:.1f} | Eps: {agent.epsilon:.3f}")

print("Training finished.")

Training:  15%|█▍        | 73/500 [00:00<00:00, 728.84it/s]

Ep 10/500 | Return: -67.0 | 50-ep avg: -63.8 | Eps: 0.993
Ep 20/500 | Return: -66.0 | 50-ep avg: -64.1 | Eps: 0.986
Ep 30/500 | Return: -67.0 | 50-ep avg: -64.5 | Eps: 0.979
Ep 40/500 | Return: -66.0 | 50-ep avg: -64.7 | Eps: 0.972
Ep 50/500 | Return: -61.0 | 50-ep avg: -64.4 | Eps: 0.965
Ep 60/500 | Return: -66.0 | 50-ep avg: -64.6 | Eps: 0.958
Ep 70/500 | Return: -66.0 | 50-ep avg: -64.7 | Eps: 0.951
Ep 80/500 | Return: -62.0 | 50-ep avg: -64.4 | Eps: 0.944
Ep 90/500 | Return: -59.0 | 50-ep avg: -64.1 | Eps: 0.937
Ep 100/500 | Return: -64.0 | 50-ep avg: -64.3 | Eps: 0.930
Ep 110/500 | Return: -62.0 | 50-ep avg: -64.1 | Eps: 0.923
Ep 120/500 | Return: -66.0 | 50-ep avg: -63.7 | Eps: 0.916
Ep 130/500 | Return: -62.0 | 50-ep avg: -63.8 | Eps: 0.909
Ep 140/500 | Return: -65.0 | 50-ep avg: -64.0 | Eps: 0.902


Training:  29%|██▉       | 146/500 [00:01<00:04, 84.26it/s]

Ep 150/500 | Return: -64.0 | 50-ep avg: -64.1 | Eps: 0.895
Ep 160/500 | Return: -65.0 | 50-ep avg: -64.0 | Eps: 0.888
Ep 170/500 | Return: -65.0 | 50-ep avg: -64.3 | Eps: 0.881


Training:  36%|███▌      | 180/500 [00:05<00:13, 24.01it/s]

...

Ep 480/500 | Return: -56.0 | 50-ep avg: -62.0 | Eps: 0.658


Training:  98%|█████████▊| 490/500 [01:33<00:04,  2.35it/s]

Ep 490/500 | Return: -62.0 | 50-ep avg: -61.7 | Eps: 0.651


Training: 100%|██████████| 500/500 [01:38<00:00,  5.10it/s]

Ep 500/500 | Return: -60.0 | 50-ep avg: -61.3 | Eps: 0.643
Training finished.


In [5]:
save_path = "agent/flappy_dqn.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
agent.save(save_path)
print("Saved:", save_path)

Saved: agent/flappy_dqn.pth


In [6]:

env.close()
state = env.reset()

total_reward = 0.0
for _ in range(5000):
	# Greedy action for evaluation (no epsilon)
	with torch.no_grad():
		state_t = torch.from_numpy(state).float().unsqueeze(0).to(agent.cfg.device)
		q_vals = agent.q_net(state_t)
		action = int(torch.argmax(q_vals, dim=1).item())

	next_state, reward, done, info = env.step(action)
	env.render(fps=60)
	total_reward += reward
	state = next_state
	if done:
		break

print("Eval episode reward:", total_reward)
env.close()

Eval episode reward: -72.0
